In [ ]:
pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 232 kB 7.6 MB/s 


In [ ]:
import PyPDF2
import numpy as np
import pandas as pd

In [ ]:
data = {}
for order in range(1,134):
    reader = PyPDF2.PdfReader(f'/content/drive/MyDrive/Colab Notebooks/Pinnacle/ตารางชำระคำสั่งศาล/{order}.pdf')
    text = reader.pages[0].extract_text()
    data[f'{order}'] = text

In [ ]:
df = pd.DataFrame([], columns=['ลำดับ', 'เลขที่สัญญา', 'ชื่อ-สกุล', 'สาขา', 'อัตราดอกเบี้ย', 'เริ่มต้นคิดดอกเบี้ย', 'มูลหนี้ทั้งหมด'])
for order in range(1,134):
    text = data[f'{order}']

    if text.find('\n \nงวด') != -1:
        header_end = '\n \nงวด'
    elif text.find('\n\nงวด') != -1:
        header_end = '\n\nงวด'
    elif text.find('\nงวด') != -1:
        header_end = '\nงวด'
    else:
        header_end = 'งวด'
        
    header_val = []
    header = ['เลขที่สัญญา : ', 'ชื่อ-สกุล : ', 'สาขา : ', 'อัตราดอกเบี้ย : ', 'เริ่มต้นคิดดอกเบี้ย : ', 'มูลหนี้ทั้งหมด : ', header_end]
    for i in range(len(header)):
        if i == len(header)-1: break
        start_acc = text.find(header[i]) + len(header[i])
        end_acc = text.find(header[i+1])
        header_val.append(text[start_acc:end_acc])
    df.loc[order-1, 'ลำดับ'] = order
    df.loc[order-1, 1:7] = header_val[:6]
    for payment in text.split('\n')[4:-1]:
        array = np.array(payment.split(' '))
        if len(array) < 4 : continue
        item = array[0]
        val = array[[2, 5, 6, 7, 8, 9]]
        df.loc[order-1, [f'ชำระวันที่{item}', f'ชำระดอกเบี้ย{item}', f'ชำระเงินต้น{item}', f'ชำระอื่นๆ{item}', f'รวมชำระ{item}', f'ดอกเบี้ยค้าง{item}']] = val
df.to_excel('all_payment.xlsx', index=False)

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:719: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  indexer = self._get_setitem_indexer(key)
